## Creación de la base de datos desde python

Lo primero es ir a DBeaver y crearnos una base de datos llamada 'supermercados'

Después nos conectamos a la bbdd:

In [3]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import sys
import os

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import funciones_auxiliares as fa

In [3]:
conexion = fa.conexion_bbdd("supermercados")
cursor = conexion.cursor()

Creamos las tablas:

In [3]:
fa.creacion_tablas(conexion, cursor)

Inserción de datos en las tablas:

In [4]:
fa.insercion_supermercados(conexion, cursor)
fa.insercion_productos(conexion, cursor)
fa.insercion_marcas(conexion, cursor)
fa.insercion_comparativa(conexion, cursor)

# Consultas a la bbdd

In [49]:
conexion = fa.conexion_bbdd("supermercados")

Precio medio de cada producto por supermercado.

In [33]:
cursor = conexion.cursor()
cursor.execute(""" SELECT nombre
FROM supermercados s;""")
lista_supermercados = pd.DataFrame(cursor.fetchall())[0].to_list()
lista_supermercados

['alcampo', 'carrefour', 'dia', 'eroski', 'hipercor', 'mercadona']

In [36]:
cursor = conexion.cursor()
cursor.execute(""" SELECT nombre
FROM tipo_producto tp;""")
res = cursor.fetchall()
lista_productos_1 = pd.DataFrame(res)[0].to_list()
lista_productos_2 = pd.DataFrame(res)[0].str.replace(" ", "_").to_list()

['aceite_de_girasol', 'aceite_de_oliva', 'leche']

In [62]:
conexion = fa.conexion_bbdd("supermercados")
cursor = conexion.cursor()
for super in lista_supermercados:
    for i in range(len(lista_productos_1)):
        cursor = conexion.cursor()
        cursor.execute(f""" CREATE VIEW media_{super}_{lista_productos_2[i]} AS (
                            SELECT nombre_producto, round(avg(precio),2) AS precio_medio
                            FROM(SELECT c.nombre AS nombre_producto, precio
                                FROM comparativa c
                                INNER JOIN supermercados s ON c.id_supermercado = s.id_supermercado 
                                INNER JOIN tipo_producto tp ON c.id_producto = tp.id_producto
                                WHERE s.nombre = '{super}' AND tp.nombre='{lista_productos_1[i]}') AS taux
                            GROUP BY nombre_producto
                            ORDER BY precio_medio); """)
        conexion.commit()  


In [4]:
conexion = fa.conexion_bbdd("supermercados")
cursor = conexion.cursor()
cursor.execute(""" SELECT *
                    FROM media_dia_leche;""")
pd.DataFrame(cursor.fetchall()).head(2)

,0,1
0,leche semidesnatada central lechera asturiana ...,0.70
1,leche desnatada dia lactea brik 1 l,0.79
